# pGrep README (via CMDline map reduce)

Please run all cells to get the final output (i.e., safe shell scripts first; then run map-reduce ; review output

In [1]:
!pwd

/home/cloudera/share


# get a sample text file

In [2]:
!curl https://www.dropbox.com/s/z4c3ubvhmbn062h/LICENSE > LICENSE
# !wget  https://www.dropbox.com/s/z4c3ubvhmbn062h/LICENSE

curl: (6) Couldn't resolve host 'www.dropbox.com'


In [6]:
!head LICENSE

# Mapper for grep 

In [ ]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
count = 0
filename = sys.argv[2]
findword = sys.argv[1]
with open (filename, "r") as myfile:
    for line in myfile.readlines():
        if findword.lower() in line.lower():
            count = count + 1
print count

In [ ]:
!chmod a+x mapper.py

# Reducer for grep

In [ ]:
%%writefile reducer.py
#!/usr/bin/python
import sys
sum = 0
for countStr in sys.stdin:
    sum = sum + int(countStr)
print sum

In [ ]:
!chmod a+x reducer.py

# Map-Reduce framework in the Command line

In [ ]:
%%writefile pGrepCount.sh
ORIGINAL_FILE=$1
FIND_WORD=$2
BLOCK_SIZE=$3
CHUNK_FILE_PREFIX=$ORIGINAL_FILE.split
SORTED_CHUNK_FILES=$CHUNK_FILE_PREFIX*.sorted
usage()
{
    echo Parallel grep
    echo usage: pGrepCount filename word chuncksize
    echo greps file file1 in $ORIGINAL_FILE and counts the number of lines
    echo Note: file1 will be split in chunks up to $ BLOCK_SIZE chunks each
    echo $FIND_WORD each chunk will be grepCounted in parallel
}
#Splitting $ORIGINAL_FILE INTO CHUNKS
split -b $BLOCK_SIZE $ORIGINAL_FILE $CHUNK_FILE_PREFIX
#DISTRIBUTE
for file in $CHUNK_FILE_PREFIX*
do
    #grep -i $FIND_WORD $file|wc -l >$file.intermediateCount &
    ./mapper.py $FIND_WORD $file >$file.intermediateCount &
done
wait
#MERGEING INTERMEDIATE COUNT CAN TAKE THE FIRST COLUMN AND TOTOL...
#numOfInstances=$(cat *.intermediateCount | cut -f 1 | paste -sd+ - |bc)
numOfInstances=$(cat *.intermediateCount | ./reducer.py)
echo "found [$numOfInstances] occurences of [$FIND_WORD] in the file [$ORIGINAL_FILE]"

#Run the parallel grep using python-based mapper and reducer

In [ ]:
!chmod a+x pGrepCount.sh

Usage: usage: pGrepCount filename word chuncksize

In [ ]:
!./pGrepCount.sh License COPYRIGHT 4k

# Map-Reduce framework in the Command line

In [ ]:
%%writefile cmdLineMapReduce.sh
ORIGINAL_FILE=$1
BLOCK_SIZE=$2
MapperParameters=$*   #note $* will include the first and second parameter also
CHUNK_FILE_PREFIX=$ORIGINAL_FILE.split
SORTED_CHUNK_FILES=$CHUNK_FILE_PREFIX*.sorted
usage()
{
    echo Command Line Map Reduce
    echo usage: cmdLineMapReduce filename chuncksize mapperParameters
    echo Note: file1 will be split in chunks up to $ BLOCK_SIZE chunks each
    echo NOTE: please make sure the output of the mapper is what the reducer expects
}
#Splitting $ORIGINAL_FILE INTO CHUNKS
split -b $BLOCK_SIZE $ORIGINAL_FILE $CHUNK_FILE_PREFIX
#DISTRIBUTE
for file in $CHUNK_FILE_PREFIX*
do
    #grep -i $FIND_WORD $file|wc -l >$file.intermediateCount &
    ./mapper.py $MapperParameters $file >$file.intermediateCount &
done
wait
#MERGEING INTERMEDIATE COUNT CAN TAKE THE FIRST COLUMN AND TOTOL...
#numOfInstances=$(cat *.intermediateCount | cut -f 1 | paste -sd+ - |bc)
reducerOutput=$(cat *.intermediateCount | ./reducer.py)
echo "$reducerOutput"